In [24]:
import translators as ts
import json
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from tqdm import tqdm
import os
import time
import pandas as pd
tokenizer = nltk.RegexpTokenizer(r"\w+")
ps = PorterStemmer()
ts.google("I love Elephants!", "en", "bn")
langs = list(ts._google.language_map.keys())
bad_langs = ['en', 'ay', 'bm', 'doi', 'dv', 'ee', 'lus', 'mni-Mtei']
for x in bad_langs:
    langs.remove(x)

In [3]:
loc = "../refining_seriously/"
def namestr(obj, namespace = globals()):
    return [name for name in namespace if namespace[name] is obj][0]

def print_json(query, n = 3, m = 5, k=6):
    n = 3
    print(f"{namestr(query)} = "+"{\n", end="")  # start of the json
    l = sorted(list(query.keys()),
            key=lambda x: int(x[k:]))
    for QID in l[:n]:
        print('\t"'+QID+'":', query[QID][:m], "\b\b, ......],")
    for i in range(2):
        print("\t...")
    for QID in l[-n:]:
        print('\t"'+QID+'":', query[QID][:m], "\b\b, ......],")
    print("}")  # end of the json

def process(string,
            tokenizer = nltk.RegexpTokenizer(r"\w+"),
            ps = PorterStemmer(),
            stopwords = stopwords.words('english')):
    '''
    - A function to process a string and return a list of tokens.
    - We tokenize the string, remove stopwords and numbers, and
        finally stem the tokens to keep them in a list.
    - This function will be used in all cases uniformly so that 
        we can compare "APPLES WITH APPLES".
    '''
    string = tokenizer.tokenize(string.lower()) # tokenize
    tokens = [ps.stem(fl) for fl in string # stem tokens
                if not fl.isnumeric() and # remove numbers
                fl not in stopwords] # remove stopwords
    return tokens # takes string as input and returns a list

def augment(string, toLang, threshold = 5000):
    string = string.split("\n")
    text = []

    for line in string:
        if len(line)>threshold:
            text += line.split(".")
        elif line != "":
            text.append(line)

    ans = []
    for tex in tqdm(text):
        oth = ts.google(tex, "en", to)
        eng = ts.google(oth, toLang, "en")
        ans.append(eng)

    return process(". ".join(ans))

In [10]:
with open(loc+"cases.json") as f:
    prior_cases = json.load(f)
with open(loc+"Query_doc.json") as f:
    query = json.load(f)
with open(loc+"answers.json") as f:
    answers = json.load(f)

print_json(prior_cases, k=1)
# print_json(query)
# print_json(answers, 3, 1)

prior_cases = {
	"C1": ['masud', 'khan', 'v', 'state', 'uttar', ......],
	"C2": ['indian', 'oil', 'corpor', 'v', 'nepc', ......],
	"C3": ['gurpal', 'singh', 'v', 'state', 'punjab', ......],
	...
	...
	"C2912": ['dharangadhara', 'chemic', 'work', 'limit', 'v', ......],
	"C2913": ['central', 'bank', 'india', 'v', 'sethumadhavan', ......],
	"C2914": ['som', 'raj', 'soma', 'v', 'state', ......],
}


In [ ]:
prior_cases_au = {}
for case in tqdm(prior_cases.keys()):
    nEnd = int(case[1:])


In [21]:
print(langs)

['af', 'ak', 'am', 'ar', 'as', 'az', 'be', 'bg', 'bho', 'bn', 'bs', 'ca', 'ceb', 'ckb', 'co', 'cs', 'cy', 'da', 'de', 'el', 'en-US', 'eo', 'es', 'et', 'eu', 'fa', 'fi', 'fr', 'fy', 'ga', 'gd', 'gl', 'gn', 'gom', 'gu', 'ha', 'haw', 'hi', 'hmn', 'hr', 'ht', 'hu', 'hy', 'id', 'ig', 'ilo', 'is', 'it', 'iw', 'ja', 'jw', 'ka', 'kk', 'km', 'kn', 'ko', 'kri', 'ku', 'ky', 'la', 'lb', 'lg', 'ln', 'lo', 'lt', 'lus', 'lv', 'mai', 'mg', 'mi', 'mk', 'ml', 'mn', 'mni-Mtei', 'mr', 'ms', 'mt', 'my', 'ne', 'nl', 'no', 'nso', 'ny', 'om', 'or', 'pa', 'pl', 'ps', 'pt', 'qu', 'ro', 'ru', 'rw', 'sa', 'sd', 'si', 'sk', 'sl', 'sm', 'sn', 'so', 'sq', 'sr', 'st', 'su', 'sv', 'sw', 'ta', 'te', 'tg', 'th', 'ti', 'tk', 'tl', 'tr', 'ts', 'tt', 'ug', 'uk', 'ur', 'uz', 'vi', 'xh', 'yi', 'yo', 'zh-CN', 'zh-TW', 'zu']


In [6]:
bad = []
tees = {}
for lang in tqdm(langs):
    try:
        tees[lang] = ts.google("I love Elephants!", "en", lang)
    except:
        bad.append(lang)

100%|██████████| 134/134 [06:08<00:00,  2.75s/it]


In [9]:
back = {}
for lang in tqdm(langs):
    back[lang] = ts.google(tees[lang], lang, "en")

100%|██████████| 134/134 [06:02<00:00,  2.71s/it]


In [23]:
bad = []
for x, y in back.items():
    if "elephant" not in y.lower():
        bad.append(x)
bad

['ay', 'bm', 'doi', 'dv', 'ee', 'lus', 'mni-Mtei']

In [16]:
back

{'af': 'I love elephants!',
 'ak': 'I’m happy with the elephant!',
 'am': 'I love elephants!',
 'ar': 'I love elephants!',
 'as': 'I love elephants!',
 'ay': 'I love the electives!',
 'az': 'I love the elephants!',
 'be': 'I love elephants!',
 'bg': 'I love elephants!',
 'bho': 'I love an elephant!',
 'bm': 'I have a great thing for me!',
 'bn': 'I love elephants!',
 'bs': 'I love elephants!',
 'ca': 'I love elephants!',
 'ceb': 'I love the elephants!',
 'ckb': 'I love the elephants!',
 'co': 'I love the elephants!',
 'cs': 'I love elephants!',
 'cy': 'I love elephants!',
 'da': 'I love elephants!',
 'de': 'I love elephants!',
 'doi': 'I love the hand!',
 'dv': 'I love male bays!',
 'ee': 'I love horses so much!',
 'el': 'I love elephants!',
 'en': 'I love Elephants!',
 'en-US': 'I love Elephants!',
 'eo': 'I love elephants!',
 'es': 'I love elephants!',
 'et': 'I love elephants!',
 'eu': 'I love elephants!',
 'fa': 'I love elephants!',
 'fi': 'I love elephants!',
 'fr': 'I love elepha

In [15]:
bad

['ay', 'bm', 'doi', 'dv', 'ee', 'en', 'en-US', 'lus', 'mni-Mtei', 'om']

In [7]:
tees

{'af': 'Ek is lief vir olifante!',
 'ak': 'M’ani gye asono ho!',
 'am': 'ዝሆኖችን እወዳለሁ!',
 'ar': 'أنا أحب الفيلة!',
 'as': 'মই হাতী ভাল পাওঁ!',
 'ay': '¡Nayaw elefantenakar wal munasta!',
 'az': 'Filləri sevirəm!',
 'be': 'Я люблю сланоў!',
 'bg': 'Обичам слонове!',
 'bho': 'हमरा हाथी से प्यार बा!',
 'bm': 'Wolo ka di n ye kosɛbɛ!',
 'bn': 'আমি হাতি ভালবাসি!',
 'bs': 'Volim slonove!',
 'ca': "M'encanten els elefants!",
 'ceb': 'Ganahan ko sa mga elepante!',
 'ckb': 'من فیلەکانم خۆشدەوێت!',
 'co': "Amu l'elefanti!",
 'cs': 'Miluji slony!',
 'cy': "Dwi'n caru eliffantod!",
 'da': 'Jeg elsker elefanter!',
 'de': 'Ich liebe Elefanten!',
 'doi': 'मैं हाथी प्यार करदा हां!',
 'dv': 'އަހަރެން ލޯބިވަނީ ފިރިހެން ފިނޮޅުތަކަށް!',
 'ee': 'Melɔ̃a sɔwo ŋutɔ!',
 'el': 'Λατρεύω τους ελέφαντες!',
 'en': 'I love Elephants!',
 'en-US': 'I love Elephants!',
 'eo': 'Mi amas elefantojn!',
 'es': '¡Amo a los elefantes!',
 'et': 'Ma armastan elevante!',
 'eu': 'Elefanteak maite ditut!',
 'fa': 'من عاشق فیل ها هس